In [1]:
from gurobipy import GRB
import gurobipy as gb

In [2]:
# Linear price response functions (intercept, slope)
snowsuit = [[80, 0.5], [80, 0.5], [30, 0.5], [30, 1.0]]
jacket = [[120, 0.7], [90, 0.9], [80, 1.0], [50, 0.9]]
snowpants = [[50, 0.8], [70, 0.4], [40, 0.4], [10, 0.4]]

In [3]:
# Create a new optimization model to maximize revenue
model = gb.Model("Markdown Optimization")

Restricted license - for non-production use only - expires 2025-11-24


In [4]:
# Construct the decision variables
p = model.addVars(3,4, lb=0, vtype=GRB.CONTINUOUS, name="Price")
d = model.addVars(3,4, lb=0, vtype=GRB.INTEGER, name="Month Demand")

In [5]:
#Objective Function
model.setObjective(gb.quicksum(p[i,n]*d[i,n] for i in range(3) for n in range(4)), GRB.MAXIMIZE)

In [6]:
# Define the demand constraints
for n in range(4):
    model.addConstr(d[0,n] == snowsuit[n][0] - snowsuit[n][1]*p[0,n], "Demand Definition Snowsuits")
    model.addConstr(d[1,n] == jacket[n][0] - jacket[n][1]*p[1,n], "Demand Definition Jackets")
    model.addConstr(d[2,n] == snowpants[n][0] - snowpants[n][1]*p[2,n], "Demand Definition Snow Pants")

In [7]:
# Demand must not exceed the number we have in stock
model.addConstr(gb.quicksum(d[0,n] + d[1,n] for n in range(4)) <= 160, "Demand Constraint 1")
model.addConstr(gb.quicksum(d[0,n] + d[2,n] for n in range(4)) <= 160, "Demand Constraint 2")   

<gurobi.Constr *Awaiting Model Update*>

In [8]:
# Prices must be marked down month-over-month
model.addConstrs((p[i,n] <= p[i,n-1] for i in range(3) for n in range(1,4)), "Markdown Constraint")

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>}

In [9]:
# Solve our model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 23 rows, 24 columns and 58 nonzeros
Model fingerprint: 0xd4aff2d7
Model has 12 quadratic objective terms
Variable types: 12 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-01, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
Found heuristic solution: objective 2812.5000000
Presolve time: 0.00s
Presolved: 48 rows, 37 columns, 121 nonzeros
Presolved model has 12 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 20 continuous, 17 integer (0 binary)

Root relaxation: objective 2.931466e+04, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bo

In [10]:
# Price of snowsuits 
print("Snowsuit Prices from January to April: \n", ['%.2f' % p[0,n].x for n in range(4)])
print("Jacket Prices from January to April: \n", ['%.2f' % p[1,n].x for n in range(4)])
print("Snow Pant Prices from January to April: \n", ['%.2f' % p[2,n].x for n in range(4)])

Snowsuit Prices from January to April: 
 ['102.00', '100.00', '50.00', '30.00']
Jacket Prices from January to April: 
 ['107.14', '71.11', '61.00', '48.89']
Snow Pant Prices from January to April: 
 ['50.00', '50.00', '50.00', '12.50']
